In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models

data = pd.read_csv('blackhole.csv')
print(f'Dataset shape: {data.shape}')
data.head()


Dataset shape: (404134, 18)


,time,source,destination,length,info,transmission_rate_per_1000_ms,reception_rate_per_1000_ms,transmission_average_per_sec,reception_average_per_sec,transmission_count_per_sec,reception_count_per_sec,transmission_total_duration_per_sec,reception_total_duration_per_sec,dao,dis,dio,category,label
0,0.037,39,9999,0.0,1.0,0.000000,0.671176,0.000000,0.499879,0.000000,0.671176,0.539313,0.570032,0.0,0.0,0.000000,Normal,0
1,0.037,39,9999,0.0,1.0,0.000000,0.649873,0.000000,0.505234,0.000000,0.649873,0.264704,0.530547,0.0,0.0,0.000000,Normal,0
2,0.038,39,9999,0.0,1.0,0.671176,0.652361,0.462516,0.501327,0.671768,0.652361,0.546376,1.000000,0.0,0.0,0.690115,Blackhole,1
3,0.045,39,9999,0.0,1.0,0.000000,0.633786,0.000000,0.517346,0.000000,0.634105,0.585425,0.553276,0.0,0.0,0.000000,Normal,0
4,0.046,39,9999,0.0,1.0,0.000000,0.630378,0.000000,0.538789,0.000000,0.630378,0.443171,0.615377,0.0,0.0,0.000000,Normal,0


In [3]:
normal = data[data['label'] == 0]
blackhole = data[data['label'] == 1]

# Upsample minority class (blackhole)
upsampled = resample(blackhole, 
                               replace=True,    # sample with replacement
                               n_samples=len(normal), # match number in majority class
                               random_state=42) # reproducible results

# Combine majority and upsampled minority
# Identify and inspect categorical columns
print(upsampled.dtypes)

# If there are any categorical columns, convert them to numerical using one-hot encoding
upsampled_encoded = pd.get_dummies(upsampled, drop_first=True)

# Preprocessing: feature scaling after encoding
X = upsampled_encoded.drop('label', axis=1)
y = upsampled_encoded['label']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


time                                   float64
source                                   int64
destination                              int64
length                                 float64
info                                   float64
transmission_rate_per_1000_ms          float64
reception_rate_per_1000_ms             float64
transmission_average_per_sec           float64
reception_average_per_sec              float64
transmission_count_per_sec             float64
reception_count_per_sec                float64
transmission_total_duration_per_sec    float64
reception_total_duration_per_sec       float64
dao                                    float64
dis                                    float64
dio                                    float64
category                                object
label                                    int64
dtype: object


In [6]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# Siamese Network for Metric-Based Meta-Learning
class SiameseNetwork(tf.keras.Model):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.fc1 = layers.Dense(64, activation='relu')
        self.fc2 = layers.Dense(32, activation='relu')
        self.out = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.fc2(x)
        return self.out(x)

# Model training setup for Siamese Network
# Model training setup for Siamese Network
def siamese_train_step(model, optimizer, X1, X2, y):
    with tf.GradientTape() as tape:
        # Forward pass for both inputs
        predictions1 = model(X1)
        predictions2 = model(X2)
        
        # Calculate the distance between the two predictions
        distance = tf.abs(predictions1 - predictions2)

        # Squeeze to ensure the shape is correct and reshape
        distance = tf.reshape(distance, [-1, 1])  # Ensure shape is (batch_size, 1)
        y = tf.reshape(y, [-1, 1])  # Ensure target shape is also (batch_size, 1)
        
        # Loss is binary cross-entropy based on distance
        loss = tf.keras.losses.binary_crossentropy(y, distance)
    
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss


# Federated Learning setup remains unchanged
def federated_train(X_splits, y_splits, num_rounds=10):
    global_model = SiameseNetwork()  # Use the Siamese model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for round_num in range(num_rounds):
        print(f"Round {round_num + 1}")
        for X, y in zip(X_splits, y_splits):
            # Split the input data into pairs for the Siamese Network
            # Here we assume X is structured as pairs for demonstration
            X1, X2 = X[:, :X.shape[1] // 2], X[:, X.shape[1] // 2:]  # Example pair splitting
            siamese_train_step(global_model, optimizer, X1, X2, y)

    return global_model

# Simulate distributed datasets for federated learning
X_train = np.random.rand(1000, 18)  # Example input data with 18 features
y_train = np.random.randint(0, 2, size=(1000, 1))  # Example binary labels

X_splits = np.array_split(X_train, 5)  # Split data for 5 clients
y_splits = np.array_split(y_train, 5)

# Federated training
global_model = federated_train(X_splits, y_splits)


Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10


In [10]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Siamese Network for Metric-Based Meta-Learning
class SiameseNetwork(tf.keras.Model):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.fc1 = layers.Dense(64, activation='relu')
        self.fc2 = layers.Dense(32, activation='relu')
        self.out = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.fc2(x)
        return self.out(x)

# Model training setup for Siamese Network
def siamese_train_step(model, optimizer, X1, X2, y):
    with tf.GradientTape() as tape:
        # Forward pass for both inputs
        predictions1 = model(X1)
        predictions2 = model(X2)

        # Calculate the distance between the two predictions
        distance = tf.abs(predictions1 - predictions2)

        # Ensure correct shape
        distance = tf.reshape(distance, [-1, 1])  # Ensure shape is (batch_size, 1)
        y = tf.reshape(y, [-1, 1])  # Ensure target shape is also (batch_size, 1)

        # Loss is binary cross-entropy based on distance
        loss = tf.keras.losses.binary_crossentropy(y, distance)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Federated Learning setup remains unchanged
def federated_train(X_splits, y_splits, num_rounds=10):
    global_model = SiameseNetwork()  # Use the Siamese model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for round_num in range(num_rounds):
        print(f"Round {round_num + 1}")
        for X, y in zip(X_splits, y_splits):
            # Split the input data into pairs for the Siamese Network
            # Here we assume X is structured as pairs for demonstration
            X1, X2 = X[:, :X.shape[1] // 2], X[:, X.shape[1] // 2:]  # Example pair splitting
            siamese_train_step(global_model, optimizer, X1, X2, y)

    return global_model

# Simulate distributed datasets for federated learning
X_train = np.random.rand(1000, 18)  # Example input data with 18 features
y_train = np.random.randint(0, 2, size=(1000, 1))  # Example binary labels

X_splits = np.array_split(X_train, 5)  # Split data for 5 clients
y_splits = np.array_split(y_train, 5)

# Federated training
global_model = federated_train(X_splits, y_splits)

# Inference on test data
def create_pairs(X):
    # Assuming X is structured correctly for Siamese network pairing
    # For simplicity, we use pairs of random slices
    X1 = X[:, :X.shape[1] // 2]  # First half
    X2 = X[:, X.shape[1] // 2:]  # Second half
    return X1, X2

# Prepare a test set
X_test = np.random.rand(200, 18)  # Example test data with 18 features
y_test = np.random.randint(0, 2, size=(200, 1))  # Example binary labels for test

# Create pairs for testing
X1_test, X2_test = create_pairs(X_test)

# Make predictions
predictions1 = global_model(X1_test)
predictions2 = global_model(X2_test)

# Calculate the absolute distance
distance = tf.abs(predictions1 - predictions2)
y_pred = tf.round(tf.squeeze(distance))  # Convert to binary predictions

# Evaluate
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))


Round 1
Round 2
Round 3
Round 4
Round 5
Round 6
Round 7
Round 8
Round 9
Round 10
Confusion Matrix:
[[ 90   0]
 [110   0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.45      1.00      0.62        90
           1       0.00      0.00      0.00       110

    accuracy                           0.45       200
   macro avg       0.23      0.50      0.31       200
weighted avg       0.20      0.45      0.28       200


Accuracy Score:
0.45


C:\Users\rathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\rathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\rathi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo